In [42]:
import pandas as pd
import numpy as np
from collections import defaultdict
import pickle
import os

class FantasyFootballDraftAssistant:
    def __init__(self):
        self.df = None
        self.positions = {'QB': (1, 2), 'RB': (6, 9), 'WR': (5, 9), 'TE': (1, 2), 'K': (1, 1), 'DST': (1, 1)}
        self.starting_positions = {'QB': 1, 'RB': 2, 'WR': 3, 'TE': 1, 'K': 1, 'DST': 1, 'FLEX': 1}
        self.flex_positions = ['RB', 'WR', 'TE']
        self.q_table = {}
        self.alpha = 0.1  # Learning rate
        self.gamma = 0.9  # Discount factor
        self.epsilon = 0.1  # Exploration rate
        self.total_episodes = 0
        
    def load_data(self, file_path):
        self.df = pd.read_csv(file_path)
        self.df['ADP'] = pd.to_numeric(self.df['ADP'], errors='coerce')
        self.df = self.df.sort_values('ADP').reset_index(drop=True)
        self.players = self.df.to_dict('records')
        
    def get_state(self, team, round_num):
        return (tuple(sum(1 for p in team if p['pos'] == pos) for pos in self.positions.keys()), round_num)
    
    def get_actions(self, available_players, team, round_num):
        actions = []
        for player in available_players:
            pos = player['pos']
            pos_count = sum(1 for p in team if p['pos'] == pos)
            if pos_count < self.positions[pos][1] and (pos not in ['K', 'DST'] or round_num > 12):
                actions.append(player['player'])
        return actions
    
    def calculate_team_value(self, team):
        weekly_scores = []
        for week in range(1, 18):  # 17-week season
            available_players = [p for p in team if p['bye_week'] != week]
            starters = self.select_starters(available_players)
            week_score = sum(p['ppr_projection'] / 17 for p in starters)
            weekly_scores.append(week_score)
        
        total_score = sum(weekly_scores)
        bench_strength = sum(p['ppr_projection'] for p in team) - sum(p['ppr_projection'] for p in self.select_starters(team))
        return total_score + (bench_strength * 0.1)  # Adding some value for bench strength
    
    def select_starters(self, available_players):
        starters = []
        for pos, count in self.starting_positions.items():
            if pos == 'FLEX':
                flex_options = sorted([p for p in available_players if p['pos'] in self.flex_positions and p not in starters], 
                                      key=lambda x: x['ppr_projection'], reverse=True)
                starters.extend(flex_options[:count])
            else:
                pos_players = sorted([p for p in available_players if p['pos'] == pos and p not in starters], 
                                     key=lambda x: x['ppr_projection'], reverse=True)
                starters.extend(pos_players[:count])
        return starters
    
    def calculate_reward(self, team, player, round_num):
        team_value_before = self.calculate_team_value(team)
        new_team = team + [player]
        team_value_after = self.calculate_team_value(new_team)
        value_added = team_value_after - team_value_before
        
        adp_bonus = max(0, (200 - player['ADP']) / 10) if not np.isnan(player['ADP']) else 0
        round_penalty = max(0, round_num - 12) * 5 if player['pos'] in ['K', 'DST'] else 0
        
        # Encourage early RB and WR picks
        if round_num <= 2 and player['pos'] in ['RB', 'WR']:
            value_added *= 1.2
        
        return value_added + adp_bonus - round_penalty
   
    
    def recommend_players(self, team, available_players, round_num, num_recommendations=5):
        state = self.get_state(team, round_num)
        actions = self.get_actions(available_players, team, round_num)
        q_values = [(a, self.q_table.get((state, a), 0)) for a in actions]  # Change this line
        top_actions = sorted(q_values, key=lambda x: x[1], reverse=True)[:num_recommendations]
        return [(next(p for p in available_players if p['player'] == a), q) for a, q in top_actions]
    
    def simulate_draft(self, user_position, num_teams=12):
        teams = [[] for _ in range(num_teams)]
        available_players = self.players.copy()
        user_picks = []
        
        for round_num in range(1, 19):
            draft_order = range(num_teams) if round_num % 2 == 1 else reversed(range(num_teams))
            for pick_in_round, team_index in enumerate(draft_order):
                if team_index == user_position:
                    recommendations = self.recommend_players(teams[team_index], available_players, round_num)
                    user_picks.append(((round_num - 1) * num_teams + pick_in_round + 1, recommendations))
                    
                    if recommendations:
                        selected_player, _ = recommendations[0]
                        teams[team_index].append(selected_player)
                        available_players = [p for p in available_players if p['player'] != selected_player['player']]
                else:
                    # Other teams draft based on ADP
                    valid_players = [p for p in available_players if sum(1 for player in teams[team_index] if player['pos'] == p['pos']) < self.positions[p['pos']][1]]
                    if valid_players:
                        player = min(valid_players, key=lambda p: p['ADP'] if not np.isnan(p['ADP']) else float('inf'))
                        teams[team_index].append(player)
                        available_players = [p for p in available_players if p['player'] != player['player']]
        
        return user_picks, teams

    def train(self, num_episodes=10000, num_teams=12):
        start_episode = self.total_episodes
        for episode in range(start_episode, start_episode + num_episodes):
            if episode % 1000 == 0:
                print(f"Training episode {episode}/{start_episode + num_episodes}")
            
            teams = [[] for _ in range(num_teams)]
            available_players = self.players.copy()
            
            for round_num in range(1, 19):
                for team_index in range(num_teams):
                    state = self.get_state(teams[team_index], round_num)
                    actions = self.get_actions(available_players, teams[team_index], round_num)
                    
                    if not actions:
                        continue
                    
                    if np.random.uniform(0, 1) < self.epsilon:
                        action = np.random.choice(actions)
                    else:
                        q_values = [self.q_table.get((state, a), 0) for a in actions]  # Change this line
                        action = actions[np.argmax(q_values)]
                    
                    player = next(p for p in available_players if p['player'] == action)
                    reward = self.calculate_reward(teams[team_index], player, round_num)
                    teams[team_index].append(player)
                    available_players = [p for p in available_players if p['player'] != action]
                    
                    next_state = self.get_state(teams[team_index], round_num + 1)
                    
                    # Q-value update
                    max_future_q = max([self.q_table.get((next_state, a), 0) for a in self.get_actions(available_players, teams[team_index], round_num + 1)], default=0)
                    current_q = self.q_table.get((state, action), 0)  # Change this line
                    new_q = (1 - self.alpha) * current_q + self.alpha * (reward + self.gamma * max_future_q)
                    self.q_table[(state, action)] = new_q  # Change this line
        
        self.total_episodes += num_episodes
    
    def save_model(self, file_path):
        with open(file_path, 'wb') as f:
            pickle.dump((dict(self.q_table), self.total_episodes), f)  # Change this line
        print(f"Model saved to {file_path}")
    
    def load_model(self, file_path):
        with open(file_path, 'rb') as f:
            loaded_q_table, self.total_episodes = pickle.load(f)
            self.q_table = loaded_q_table  # Change this line
        print(f"Model loaded from {file_path}. Total episodes: {self.total_episodes}")




In [59]:
# Usage and testing
assistant = FantasyFootballDraftAssistant()
assistant.load_data('data//cbs_fantasy_projection_master.csv')

model_file = 'data//fantasy_football_model.pkl'

# Check if a saved model exists
if os.path.exists(model_file):
    assistant.load_model(model_file)
    print("Continuing training from saved model...")
else:
    print("Starting new training...")

# Train for 10,000 episodes
print("Training the model...")
assistant.train(num_episodes=50000)

# Save the model after training
assistant.save_model(model_file)


Model loaded from data//fantasy_football_model.pkl. Total episodes: 60000
Continuing training from saved model...
Training the model...
Training episode 60000/110000
Training episode 61000/110000
Training episode 62000/110000
Training episode 63000/110000
Training episode 64000/110000
Training episode 65000/110000
Training episode 66000/110000
Training episode 67000/110000
Training episode 68000/110000
Training episode 69000/110000
Training episode 70000/110000
Training episode 71000/110000
Training episode 72000/110000
Training episode 73000/110000
Training episode 74000/110000
Training episode 75000/110000
Training episode 76000/110000
Training episode 77000/110000
Training episode 78000/110000
Training episode 79000/110000
Training episode 80000/110000
Training episode 81000/110000
Training episode 82000/110000
Training episode 83000/110000
Training episode 84000/110000
Training episode 85000/110000
Training episode 86000/110000
Training episode 87000/110000
Training episode 88000/1

In [60]:


user_position=7
print(f"\nSimulating a draft for a user in position {user_position + 1}...")
user_picks, simulated_teams = assistant.simulate_draft(user_position=user_position)  # 0-based index

print("\nRecommendations for your picks:")
for pick_number, recommendations in user_picks:
    print(f"\nPick {pick_number}:")
    for i, (player, q_value) in enumerate(recommendations, 1):
        print(f"  {i}. {player['player']} ({player['pos']}) - Q-value: {q_value:.2f}, ADP: {player['ADP']:.2f}")

print("\nYour team:")
for player in simulated_teams[user_position]:  # user_position is 2 (0-based index)
    print(f"{player['player']} ({player['pos']}) - ADP: {player['ADP']:.2f}, Projection: {player['ppr_projection']:.2f}")

# Analyze position distribution for all teams
def analyze_team(team):
    positions = {pos: 0 for pos in assistant.positions.keys()}
    for player in team:
        positions[player['pos']] += 1
    return positions

print("\nPosition distribution for all teams:")
for i, team in enumerate(simulated_teams):
    positions = analyze_team(team)
    print(f"Team {i+1}: {positions}")

# Calculate and print total projected points for each team
print("\nTotal projected points for each team:")
for i, team in enumerate(simulated_teams):
    total_points = sum(player['ppr_projection'] for player in team)
    print(f"Team {i+1}: {total_points:.2f}")


Simulating a draft for a user in position 8...

Recommendations for your picks:

Pick 8:
  1. Amon-Ra St. Brown (WR) - Q-value: 258.90, ADP: 6.80
  2. A.J. Brown (WR) - Q-value: 256.85, ADP: 10.20
  3. Puka Nacua (WR) - Q-value: 256.71, ADP: 12.40
  4. Marvin Harrison Jr. (WR) - Q-value: 255.14, ADP: 18.00
  5. Garrett Wilson (WR) - Q-value: 253.97, ADP: 13.60

Pick 17:
  1. Davante Adams (WR) - Q-value: 241.69, ADP: 18.40
  2. Chris Olave (WR) - Q-value: 240.57, ADP: 23.40
  3. Drake London (WR) - Q-value: 239.99, ADP: 25.20
  4. Nico Collins (WR) - Q-value: 239.81, ADP: 32.80
  5. Deebo Samuel (WR) - Q-value: 239.52, ADP: 33.00

Pick 32:
  1. Lamar Jackson (QB) - Q-value: 224.49, ADP: 37.60
  2. C.J. Stroud (QB) - Q-value: 222.15, ADP: 47.60
  3. Joe Burrow (QB) - Q-value: 221.51, ADP: 60.00
  4. Anthony Richardson (QB) - Q-value: 221.18, ADP: 53.40
  5. Dak Prescott (QB) - Q-value: 220.78, ADP: 68.40

Pick 41:
  1. Mark Andrews (TE) - Q-value: 202.84, ADP: 43.60
  2. Trey McBride (

In [54]:


user_position=0
print(f"\nSimulating a draft for a user in position {user_position}...")
user_picks, simulated_teams = assistant.simulate_draft(user_position=user_position)  # 0-based index

print("\nRecommendations for your picks:")
for pick_number, recommendations in user_picks:
    print(f"\nPick {pick_number}:")
    for i, (player, q_value) in enumerate(recommendations, 1):
        print(f"  {i}. {player['player']} ({player['pos']}) - Q-value: {q_value:.2f}, ADP: {player['ADP']:.2f}")

print("\nYour team:")
for player in simulated_teams[user_position]:  # user_position is 2 (0-based index)
    print(f"{player['player']} ({player['pos']}) - ADP: {player['ADP']:.2f}, Projection: {player['ppr_projection']:.2f}")

# Analyze position distribution for all teams
def analyze_team(team):
    positions = {pos: 0 for pos in assistant.positions.keys()}
    for player in team:
        positions[player['pos']] += 1
    return positions

print("\nPosition distribution for all teams:")
for i, team in enumerate(simulated_teams):
    positions = analyze_team(team)
    print(f"Team {i+1}: {positions}")

# Calculate and print total projected points for each team
print("\nTotal projected points for each team:")
for i, team in enumerate(simulated_teams):
    total_points = sum(player['ppr_projection'] for player in team)
    print(f"Team {i+1}: {total_points:.2f}")


Simulating a draft for a user in position 0...

Recommendations for your picks:

Pick 1:
  1. CeeDee Lamb (WR) - Q-value: 257.48, ADP: 2.00
  2. Tyreek Hill (WR) - Q-value: 255.71, ADP: 3.20
  3. Amon-Ra St. Brown (WR) - Q-value: 254.56, ADP: 6.80
  4. Justin Jefferson (WR) - Q-value: 254.08, ADP: 6.00
  5. Ja'Marr Chase (WR) - Q-value: 252.79, ADP: 5.80

Pick 24:
  1. Drake London (WR) - Q-value: 235.08, ADP: 25.20
  2. Nico Collins (WR) - Q-value: 234.90, ADP: 32.80
  3. Deebo Samuel (WR) - Q-value: 234.61, ADP: 33.00
  4. Mike Evans (WR) - Q-value: 234.34, ADP: 30.80
  5. Cooper Kupp (WR) - Q-value: 234.11, ADP: 38.20

Pick 25:
  1. Jalen Hurts (QB) - Q-value: 223.78, ADP: 31.80
  2. Josh Allen (QB) - Q-value: 223.31, ADP: 26.20
  3. Patrick Mahomes (QB) - Q-value: 221.06, ADP: 29.60
  4. Lamar Jackson (QB) - Q-value: 219.69, ADP: 37.60
  5. C.J. Stroud (QB) - Q-value: 217.10, ADP: 47.60

Pick 48:
  1. George Kittle (TE) - Q-value: 196.57, ADP: 59.00
  2. David Njoku (TE) - Q-value

In [36]:
# Usage and testing
assistant = FantasyFootballDraftAssistant()
assistant.load_data('data//cbs_fantasy_projection_master.csv')

model_file = 'fantasy_football_model.pkl'

# Check if a saved model exists
if os.path.exists(model_file):
    assistant.load_model(model_file)
    print("Continuing training from saved model...")
else:
    print("Starting new training...")

# Train for 10,000 episodes
print("Training the model...")
assistant.train(num_episodes=10000)

# Save the model after training
#assistant.save_model(model_file)

print("\nSimulating a draft for a user in position 3...")
user_picks, simulated_teams = assistant.simulate_draft(user_position=2)  # 0-based index

print("\nRecommendations for your picks:")
for pick_number, recommendations in user_picks:
    print(f"\nPick {pick_number}:")
    for i, (player, q_value) in enumerate(recommendations, 1):
        print(f"  {i}. {player['player']} ({player['pos']}) - Q-value: {q_value:.2f}, ADP: {player['ADP']:.2f}")

print("\nYour team:")
for player in simulated_teams[2]:  # user_position is 2 (0-based index)
    print(f"{player['player']} ({player['pos']}) - ADP: {player['ADP']:.2f}, Projection: {player['ppr_projection']:.2f}")

# Analyze position distribution for all teams
def analyze_team(team):
    positions = {pos: 0 for pos in assistant.positions.keys()}
    for player in team:
        positions[player['pos']] += 1
    return positions

print("\nPosition distribution for all teams:")
for i, team in enumerate(simulated_teams):
    positions = analyze_team(team)
    print(f"Team {i+1}: {positions}")

# Calculate and print total projected points for each team
print("\nTotal projected points for each team:")
for i, team in enumerate(simulated_teams):
    total_points = sum(player['ppr_projection'] for player in team)
    print(f"Team {i+1}: {total_points:.2f}")

Starting new training...
Training the model...
Training episode 0/10000
Training episode 1000/10000
Training episode 2000/10000
Training episode 3000/10000
Training episode 4000/10000
Training episode 5000/10000
Training episode 6000/10000
Training episode 7000/10000
Training episode 8000/10000
Training episode 9000/10000


AttributeError: Can't pickle local object 'FantasyFootballDraftAssistant.__init__.<locals>.<lambda>'

In [37]:

print("\nSimulating a draft for a user in position 3...")
user_picks, simulated_teams = assistant.simulate_draft(user_position=2)  # 0-based index

print("\nRecommendations for your picks:")
for pick_number, recommendations in user_picks:
    print(f"\nPick {pick_number}:")
    for i, (player, q_value) in enumerate(recommendations, 1):
        print(f"  {i}. {player['player']} ({player['pos']}) - Q-value: {q_value:.2f}, ADP: {player['ADP']:.2f}")

print("\nYour team:")
for player in simulated_teams[2]:  # user_position is 2 (0-based index)
    print(f"{player['player']} ({player['pos']}) - ADP: {player['ADP']:.2f}, Projection: {player['ppr_projection']:.2f}")

# Analyze position distribution for all teams
def analyze_team(team):
    positions = {pos: 0 for pos in assistant.positions.keys()}
    for player in team:
        positions[player['pos']] += 1
    return positions

print("\nPosition distribution for all teams:")
for i, team in enumerate(simulated_teams):
    positions = analyze_team(team)
    print(f"Team {i+1}: {positions}")

# Calculate and print total projected points for each team
print("\nTotal projected points for each team:")
for i, team in enumerate(simulated_teams):
    total_points = sum(player['ppr_projection'] for player in team)
    print(f"Team {i+1}: {total_points:.2f}")


Simulating a draft for a user in position 3...

Recommendations for your picks:

Pick 3:
  1. Tyreek Hill (WR) - Q-value: 239.89, ADP: 3.20
  2. Amon-Ra St. Brown (WR) - Q-value: 238.73, ADP: 6.80
  3. Justin Jefferson (WR) - Q-value: 238.25, ADP: 6.00
  4. Ja'Marr Chase (WR) - Q-value: 236.96, ADP: 5.80
  5. A.J. Brown (WR) - Q-value: 236.68, ADP: 10.20

Pick 22:
  1. Alvin Kamara (RB) - Q-value: 214.24, ADP: 39.60
  2. James Conner (RB) - Q-value: 210.46, ADP: 60.40
  3. Tank Dell (WR) - Q-value: 210.03, ADP: 63.80
  4. Dalton Kincaid (TE) - Q-value: 207.45, ADP: 52.20
  5. Jordan Addison (WR) - Q-value: 204.79, ADP: 96.20

Pick 27:
  1. Deebo Samuel (WR) - Q-value: 201.82, ADP: 33.00
  2. Michael Pittman (WR) - Q-value: 201.08, ADP: 32.60
  3. Josh Jacobs (RB) - Q-value: 191.06, ADP: 30.40
  4. Javonte Williams (RB) - Q-value: 182.82, ADP: 97.20
  5. Brandon Aiyuk (WR) - Q-value: 142.17, ADP: 32.80

Pick 46:
  1. Brock Bowers (TE) - Q-value: 174.04, ADP: 102.40
  2. Amari Cooper (W

In [34]:
import pandas as pd
import numpy as np
from collections import defaultdict

class FantasyFootballDraftAssistant:
    def __init__(self):
        self.df = None
        self.positions = {'QB': (1, 3), 'RB': (3, 9), 'WR': (4, 9), 'TE': (1, 3), 'K': (1, 1), 'DST': (1, 2)}
        self.starting_positions = {'QB': 1, 'RB': 2, 'WR': 3, 'TE': 1, 'K': 1, 'DST': 1, 'FLEX': 1}
        self.flex_positions = ['RB', 'WR', 'TE']
        self.q_table = defaultdict(lambda: defaultdict(float))
        self.alpha = 0.1  # Learning rate
        self.gamma = 0.9  # Discount factor
        self.epsilon = 0.1  # Exploration rate
        
    def load_data(self, file_path):
        self.df = pd.read_csv(file_path)
        self.df['ADP'] = pd.to_numeric(self.df['ADP'], errors='coerce')
        self.df = self.df.sort_values('ppr_projection', ascending=False).reset_index(drop=True)
        self.players = self.df.to_dict('records')
        
    def get_state(self, team, round_num):
        return (tuple(sum(1 for p in team if p['pos'] == pos) for pos in self.positions.keys()), round_num)
    
    def get_actions(self, available_players, team, round_num):
        actions = []
        for player in available_players:
            pos = player['pos']
            pos_count = sum(1 for p in team if p['pos'] == pos)
            if pos_count < self.positions[pos][1] and (pos not in ['K', 'DST'] or round_num > 12):
                actions.append(player['player'])
        return actions
    
    def calculate_team_value(self, team):
        weekly_scores = []
        for week in range(1, 18):  # 17-week season
            available_players = [p for p in team if p['bye_week'] != week]
            starters = self.select_starters(available_players)
            week_score = sum(p['ppr_projection'] / 17 for p in starters)
            weekly_scores.append(week_score)
        
        total_score = sum(weekly_scores)
        bench_strength = sum(p['ppr_projection'] for p in team) - sum(p['ppr_projection'] for p in self.select_starters(team))
        return total_score + (bench_strength * 0.1)  # Adding some value for bench strength
    
    def select_starters(self, available_players):
        starters = []
        for pos, count in self.starting_positions.items():
            if pos == 'FLEX':
                flex_options = sorted([p for p in available_players if p['pos'] in self.flex_positions and p not in starters], 
                                      key=lambda x: x['ppr_projection'], reverse=True)
                starters.extend(flex_options[:count])
            else:
                pos_players = sorted([p for p in available_players if p['pos'] == pos and p not in starters], 
                                     key=lambda x: x['ppr_projection'], reverse=True)
                starters.extend(pos_players[:count])
        return starters
    
    def calculate_reward(self, team, player, round_num):
        team_value_before = self.calculate_team_value(team)
        new_team = team + [player]
        team_value_after = self.calculate_team_value(new_team)
        value_added = team_value_after - team_value_before
        
        adp_bonus = max(0, (200 - player['ADP']) / 10) if not np.isnan(player['ADP']) else 0
        round_penalty = max(0, round_num - 10) * 2 if player['pos'] in ['K', 'DST'] else 0
        
        return value_added + adp_bonus - round_penalty
    
    def train(self, num_episodes=10000, num_teams=12):
        for episode in range(num_episodes):
            if episode % 1000 == 0:
                print(f"Training episode {episode}/{num_episodes}")
            
            teams = [[] for _ in range(num_teams)]
            available_players = self.players.copy()
            
            for round_num in range(1, 19):
                for team_index in range(num_teams):
                    state = self.get_state(teams[team_index], round_num)
                    actions = self.get_actions(available_players, teams[team_index], round_num)
                    
                    if not actions:
                        continue
                    
                    if np.random.uniform(0, 1) < self.epsilon:
                        action = np.random.choice(actions)
                    else:
                        q_values = [self.q_table[state][a] for a in actions]
                        action = actions[np.argmax(q_values)]
                    
                    player = next(p for p in available_players if p['player'] == action)
                    reward = self.calculate_reward(teams[team_index], player, round_num)
                    teams[team_index].append(player)
                    available_players = [p for p in available_players if p['player'] != action]
                    
                    next_state = self.get_state(teams[team_index], round_num + 1)
                    
                    # Q-value update
                    max_future_q = max([self.q_table[next_state][a] for a in self.get_actions(available_players, teams[team_index], round_num + 1)], default=0)
                    current_q = self.q_table[state][action]
                    new_q = (1 - self.alpha) * current_q + self.alpha * (reward + self.gamma * max_future_q)
                    self.q_table[state][action] = new_q
    
    def recommend_players(self, team, available_players, round_num, num_recommendations=5):
        state = self.get_state(team, round_num)
        actions = self.get_actions(available_players, team, round_num)
        q_values = [(a, self.q_table[state][a]) for a in actions]
        top_actions = sorted(q_values, key=lambda x: x[1], reverse=True)[:num_recommendations]
        return [(next(p for p in available_players if p['player'] == a), q) for a, q in top_actions]
    
    def simulate_draft(self, user_position, num_teams=12):
        teams = [[] for _ in range(num_teams)]
        available_players = self.players.copy()
        user_picks = []
        
        for round_num in range(1, 19):
            draft_order = range(num_teams) if round_num % 2 == 1 else reversed(range(num_teams))
            for pick_in_round, team_index in enumerate(draft_order):
                if team_index == user_position:
                    recommendations = self.recommend_players(teams[team_index], available_players, round_num)
                    user_picks.append(((round_num - 1) * num_teams + pick_in_round + 1, recommendations))
                    
                    if recommendations:
                        selected_player, _ = recommendations[0]
                        teams[team_index].append(selected_player)
                        available_players = [p for p in available_players if p['player'] != selected_player['player']]
                else:
                    actions = self.get_actions(available_players, teams[team_index], round_num)
                    if actions:
                        player = max(
                            [p for p in available_players if p['player'] in actions],
                            key=lambda p: p['ppr_projection'] - (round_num * 2 if p['pos'] in ['K', 'DST'] else 0)
                        )
                        teams[team_index].append(player)
                        available_players = [p for p in available_players if p['player'] != player['player']]
        
        return user_picks, teams

# Usage and testing
assistant = FantasyFootballDraftAssistant()
assistant.load_data('data//cbs_fantasy_projection_master.csv')

print("Training the model...")
assistant.train(num_episodes=10000)  # Increase this for better results

print("\nSimulating a draft for a user in position 3...")
user_picks, simulated_teams = assistant.simulate_draft(user_position=2)  # 0-based index

print("\nRecommendations for your picks:")
for pick_number, recommendations in user_picks:
    print(f"\nPick {pick_number}:")
    for i, (player, q_value) in enumerate(recommendations, 1):
        print(f"  {i}. {player['player']} ({player['pos']}) - Q-value: {q_value:.2f}")

print("\nYour team:")
for player in simulated_teams[2]:  # user_position is 2 (0-based index)
    print(f"{player['player']} ({player['pos']}) - ADP: {player['ADP']:.2f}, Projection: {player['ppr_projection']:.2f}")

# Analyze position distribution for all teams
def analyze_team(team):
    positions = {pos: 0 for pos in assistant.positions.keys()}
    for player in team:
        positions[player['pos']] += 1
    return positions

print("\nPosition distribution for all teams:")
for i, team in enumerate(simulated_teams):
    positions = analyze_team(team)
    print(f"Team {i+1}: {positions}")

# Calculate and print total projected points for each team
print("\nTotal projected points for each team:")
for i, team in enumerate(simulated_teams):
    total_points = sum(player['ppr_projection'] for player in team)
    print(f"Team {i+1}: {total_points:.2f}")


Training the model...
Training episode 0/10000
Training episode 1000/10000
Training episode 2000/10000
Training episode 3000/10000
Training episode 4000/10000
Training episode 5000/10000
Training episode 6000/10000
Training episode 7000/10000
Training episode 8000/10000
Training episode 9000/10000

Simulating a draft for a user in position 3...

Recommendations for your picks:

Pick 3:
  1. Patrick Mahomes (QB) - Q-value: 225.39
  2. Lamar Jackson (QB) - Q-value: 224.04
  3. C.J. Stroud (QB) - Q-value: 221.75
  4. Joe Burrow (QB) - Q-value: 220.93
  5. Anthony Richardson (QB) - Q-value: 220.77

Pick 22:
  1. Tyreek Hill (WR) - Q-value: 205.07
  2. Amon-Ra St. Brown (WR) - Q-value: 204.05
  3. A.J. Brown (WR) - Q-value: 202.28
  4. Puka Nacua (WR) - Q-value: 202.13
  5. Davante Adams (WR) - Q-value: 199.64

Pick 27:
  1. Bijan Robinson (RB) - Q-value: 185.65
  2. Travis Etienne (RB) - Q-value: 178.73
  3. Derrick Henry (RB) - Q-value: 178.47
  4. Joe Mixon (RB) - Q-value: 176.22
  5. Bo

In [ ]:

# Usage and testing
assistant = FantasyFootballDraftAssistant()
assistant.load_data('cbs_fantasy_projection_master.csv')


In [33]:

print("Training the model...")
assistant.train(num_episodes=1000)  # Increase this for better results

print("\nSimulating a draft for a user in position 3...")
user_picks, simulated_teams = assistant.simulate_draft(user_position=2)  # 0-based index

print("\nRecommendations for your picks:")
for pick_number, recommendations in user_picks:
    print(f"\nPick {pick_number}:")
    for i, (player, q_value) in enumerate(recommendations, 1):
        print(f"  {i}. {player['player']} ({player['pos']}) - Q-value: {q_value:.2f}")

# Analyze final team compositions and print user's team
print("\nYour team:")
for player in simulated_teams[2]:  # user_position is 2 (0-based index)
    print(f"{player['player']} ({player['pos']})")

# Analyze position distribution for all teams
def analyze_team(team):
    positions = {pos: 0 for pos in assistant.positions.keys()}
    for player in team:
        positions[player['pos']] += 1
    return positions

print("\nPosition distribution for all teams:")
for i, team in enumerate(simulated_teams):
    positions = analyze_team(team)
    print(f"Team {i+1}: {positions}")

Training the model...
Training episode 0/1000
Training episode 10/1000
Training episode 20/1000
Training episode 30/1000
Training episode 40/1000
Training episode 50/1000
Training episode 60/1000
Training episode 70/1000
Training episode 80/1000
Training episode 90/1000
Training episode 100/1000
Training episode 110/1000
Training episode 120/1000
Training episode 130/1000
Training episode 140/1000
Training episode 150/1000
Training episode 160/1000
Training episode 170/1000
Training episode 180/1000
Training episode 190/1000
Training episode 200/1000
Training episode 210/1000
Training episode 220/1000
Training episode 230/1000
Training episode 240/1000
Training episode 250/1000
Training episode 260/1000
Training episode 270/1000
Training episode 280/1000
Training episode 290/1000
Training episode 300/1000
Training episode 310/1000
Training episode 320/1000
Training episode 330/1000
Training episode 340/1000
Training episode 350/1000
Training episode 360/1000
Training episode 370/1000
T